In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import ipywidgets
from datetime import datetime
import geopandas as gpd
from shapely import wkt
import main
import File_creation as fc

In [4]:
c = pd.read_csv('clusters.csv')

for i in ['x','y']:
    c.rename(columns={f'geometry_{i}': 'geometry'}, inplace=True)
    c[f'geometry_{i}'] = c['geometry'].apply(wkt.loads)
    c.drop('geometry', inplace=True, axis=1)
c = gpd.GeoDataFrame(c, crs='epsg:4326',geometry = 'geometry_x')
c.set_geometry('geometry_y',inplace=True)

In [19]:
x = c.ZONE.unique()
zone_list = sorted(x.tolist())

x = c.groupby(['ZONE', 'Date']).count().sort_values(by='index_x', ascending=False)
x.rename(columns={'Unnamed: 0': 'Number of times 2 crashes occured within an hour/day'}, inplace=True)
cr = x[['Number of times 2 crashes occured within an hour/day']]
cr.reset_index(inplace=True)
@ipywidgets.interact(zone = ipywidgets.SelectionSlider(options=zone_list))
def zone_hist_by_day(zone):
    zone_filter = cr["ZONE"] == zone
    print(cr[zone_filter].head(10).sort_values(by = 'Number of times 2 crashes occured within an hour/day',ascending = False))

interactive(children=(SelectionSlider(description='zone', options=(2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 16…

In [32]:
taxi_data = fc.taxi_file_setup("sampled_combined_taxi_2018_600k.csv")
taxi_data = main.datetime_conversions(taxi_data, ['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
                                 '%m/%d/%Y %I:%M:%S %p')
taxi_data = main.add_time_and_speed(taxi_data)
taxi_data = main.removeWeirdTaxiData(taxi_data)
nyc_taxi_geo = gpd.read_file('NYC_Taxi_Zones.geojson')

/Users/tzacks2/Documents/GradSchool/Spring23/Progr_Quality/IS597_final/main.py:110: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[too_long]
/Users/tzacks2/Documents/GradSchool/Spring23/Progr_Quality/IS597_final/main.py:111: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[super_fast]
/Users/tzacks2/Documents/GradSchool/Spring23/Progr_Quality/IS597_final/main.py:112: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[super_slow]


In [33]:
taxi_data

,tripID,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,tolls_amount,total_amount,trip_time,trip_time_h,avg speed
0,19061628,2018-01-01 01:00:03,2018-01-01 01:11:10,2.56,166,239,10.5,0.00,0.00,11.80,0 days 00:11:07,0.185278,13.817091
1,19061445,2018-01-01 01:00:38,2018-01-01 01:08:50,0.80,79,148,7.0,0.00,0.00,8.30,0 days 00:08:12,0.136667,5.853659
2,9350400,2018-01-01 01:00:53,2018-01-01 01:22:33,2.67,234,148,14.5,0.00,0.00,15.80,0 days 00:21:40,0.361111,7.393846
3,9350381,2018-01-01 01:00:56,2018-01-01 01:15:04,2.21,68,234,11.0,0.00,0.00,12.30,0 days 00:14:08,0.235556,9.382075
4,19061290,2018-01-01 01:01:05,2018-01-01 01:03:40,0.44,238,239,4.0,1.06,0.00,6.36,0 days 00:02:35,0.043056,10.219355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,202215,2018-12-31 00:58:13,2018-12-31 01:23:11,6.15,237,173,22.5,0.00,0.00,23.80,0 days 00:24:58,0.416111,14.779706
599996,142307,2018-12-31 12:58:18,2018-12-31 13:13:30,5.20,170,209,18.0,3.76,0.00,22.56,0 days 00:15:12,0.253333,20.526316
599997,142263,2018-12-31 12:58:37,2018-12-31 13:30:14,18.45,132,90,52.0,0.00,5.76,58.56,0 days 00:31:37,0.526944,35.013179
599998,142237,2018-12-31 12:58:46,2018-12-31 13:01:16,0.34,142,142,3.5,0.86,0.00,5.16,0 days 00:02:30,0.041667,8.160000


In [50]:
@ipywidgets.interact(zone = ipywidgets.SelectionSlider(options=zone_list))
def plot_zone_speeds(zone):
    plt.clf()
    sub1 = taxi_data[taxi_data['PULocationID'] == zone]
    sub2 = taxi_data[taxi_data['DOLocationID'] == zone]
    sub = pd.concat([sub1,sub2])
    plt.hist(sub['avg speed'], bins = 60)
    plt.xlabel('Average Speed (mph)')
    plt.ylabel('Number of trips')
    return plt.show()

interactive(children=(SelectionSlider(description='zone', options=(2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 16…

NameError: name 'main' is not defined